In [1]:
import tensorflow as tf

In [2]:
import schedule
import time
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import layers, losses, models, optimizers

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from numpy.lib.stride_tricks import sliding_window_view
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_csv('data/BTCUSDT_210701_220630.csv')
df.head()

,open_time,open,high,close,low,volume
0,2021-07-01 00:00:00,35018.0,35021.5,34985.0,34985.0,74.194
1,2021-07-01 00:01:00,34985.0,35007.0,34980.0,34980.0,40.324
2,2021-07-01 00:02:00,34980.0,34980.5,34937.5,34911.0,166.749
3,2021-07-01 00:03:00,34937.5,34958.5,34952.0,34937.0,17.209
4,2021-07-01 00:04:00,34952.0,34975.0,34960.5,34942.5,57.475


In [4]:
def preprocess_ratio(df):
    data = df.copy()
    windows = [5,10,20,60,120]
    for window in windows:
        data['close_ma{}'.format(window)] =  data['Close'].rolling(window).mean()
        data['volume_ma{}'.format(window)] = data['Volume'].rolling(window).mean()
        data['close_ema{}'.format(window)] =  data['Close'].ewm(span=window).mean()
        data['volume_ema{}'.format(window)] = data['Volume'].ewm(span=window).mean()
        data['close_ma%d_ratio' % window] = (data['Close'] - data['close_ma%d' % window]) / data['close_ma%d' % window]
        data['volume_ma%d_ratio' % window] = (data['Volume'] - data['volume_ma%d' % window]) / data['volume_ma%d' % window]
        data['close_ema%d_ratio' % window] = (data['Close'] - data['close_ema%d' % window]) / data['close_ema%d' % window]
        data['volume_ema%d_ratio' % window] = (data['Volume'] - data['volume_ema%d' % window]) / data['volume_ema%d' % window]
        
    data['open_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'open_lastclose_ratio'] = (data['Open'][1:].values - data['Close'][:-1].values) / data['Close'][:-1].values
    
    data['high_close_ratio'] = (data['High'].values - data['Close'].values) / data['Close'].values
    
    data['low_close_ratio'] = (data['Low'].values - data['Close'].values) / data['Close'].values
    
    data['close_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'close_lastclose_ratio'] = (data['Close'][1:].values - data['Close'][:-1].values) / data['Close'][:-1].values
    
    data['volume_lastvolume_ratio'] = np.zeros(len(data))
    data.loc[1:, 'volume_lastvolume_ratio'] = (data['Volume'][1:].values - data['Volume'][:-1].values) / data['Volume'][:-1].replace(to_replace=0, method='ffill').replace(to_replace=0, method='bfill').values
    
    shortema = data['Close'].ewm(span=12).mean()
    longema = data['Close'].ewm(span=26).mean()
    DIF = shortema - longema
    DEA = DIF.ewm(span=9).mean()
    data['MACD'] = 2*(DIF-DEA)
    
    delta = data['Close'].diff(1)
    delta = delta[1:]
    
    up = delta.copy()
    down = delta.copy()
    up[up<0] = 0
    down[down>0] = 0
    data['up'] = up
    data['down'] = down
    
    avg_gain = data['up'].rolling(window=14).mean()
    avg_loss = abs(data['down'].rolling(window=14).mean())
    RS = avg_gain / avg_loss
    
    RSI = 100.0-(100.0/(1.0+RS))
    data['RSI'] = RSI
    
    ndays_high=data["High"].rolling(window=14, min_periods=1).max()
    ndays_low=data['Low'].rolling(window=14, min_periods=1).min()
    fast_k = ((data["Close"]-ndays_low)/(ndays_high-ndays_low+1e-6))*100
    data["Fast_k"]=fast_k
    
    typical_price = ((df['Close'] + df['High'] + df['Low']) / 3)
    money_flow = typical_price * df['Volume']
    positive_flow = []
    negative_flow = []
    for i in range(1, len(typical_price)):
        if typical_price[i] > typical_price[i-1] :
            positive_flow.append(money_flow[i-1])
            negative_flow.append(0)
        elif typical_price[i] < typical_price[i-1]:
            negative_flow.append(money_flow[i-1])
            positive_flow.append(0)
        else:
            positive_flow.append(0)
            negative_flow.append(0)
            
    MF_period = 14
    positive_mf = [np.nan]*14
    negative_mf = [np.nan]*14
    for i in range(MF_period-1, len(positive_flow)):
        positive_mf.append(sum(positive_flow[i+1-MF_period : i+1]))
        
    for i in range(MF_period-1, len(negative_flow)):
        negative_mf.append(sum(negative_flow[i+1-MF_period : i+1]))
        
    mfi = 100 * (np.array(positive_mf) / (np.array(positive_mf) + np.array(negative_mf)))
    data['MFI'] = mfi
    
    data=data.drop(columns=['up','down'])
    data=data.dropna(axis=0).reset_index(drop=True)

    return data

In [5]:
def data_scaling(data):             #스케일러, main
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)

    return data


def data_sliding(data, n_features, Time_window): #데이터 밀기, main
    reshape_num = data.shape[0] - Time_window + 1
    data = sliding_window_view(data, (Time_window, n_features))
    data = data.reshape(reshape_num,Time_window, n_features,1)

    return data


def data_load_processing(df, Time_window=None): #데이터 로딩 및 스케일러 사용, main
    df = df.drop(['Time'], axis=1)
    n_features = df.shape[1]
    
    df = data_scaling(df)
    df = data_sliding(df, n_features, Time_window)

    return df

In [6]:
def lstm_preprocess(latent_z, seq_length):
    reshape_num = latent_z.shape[0] - seq_length + 1
    slide_z = sliding_window_view(latent_z, (seq_length, latent_z.shape[1])) #확인 필요
    slide_z = slide_z.reshape(reshape_num, seq_length, latent_z.shape[1])
    slide_z = slide_z[:-1]

    dataY = []
    for i in range(0, len(latent_z) - seq_length):
        _y = latent_z[i + seq_length]  # 다음 나타날 z(정답)
        dataY.append(_y)  # dataY 리스트에 추가
    dataY = np.array(dataY)
    
    return slide_z, dataY

In [ ]:
cnt = 0
df_120 = pd.DataFrame(columns=df.columns)
vae_model = tf.keras.models.load_model('model/vae_encoder', compile=False)
lstm_model = tf.keras.models.load_model('model/lstm', compile=False)
lstm_df = pd.DataFrame(columns=['Price','z0','z1','z2'])
save_dir=Path("log")
num_log = save_dir / "num_log"
prev_pos = 0
cash = 10000
while True:
    data = df.iloc[cnt,]
    cnt = cnt + 1
    
    if len(df_120) < 157:
        df_120.loc[len(df_120)] = data
    else:
        df_120 = df_120.drop(0).reset_index(drop=True)
        df_120.loc[len(df_120)] = data
        
        df_pre = df_120.copy()
        df_pre.columns = ["Time","Open","High","Low","Close","Volume"]
        temp = preprocess_ratio(df_pre)
        
        vae_temp = data_load_processing(temp, 8)
        _, _, z_vector=vae_model.predict(vae_temp)
        
        seq_length = 30
        slide_z, dataY = lstm_preprocess(z_vector, seq_length)
        lstm_vector = lstm_model.predict(slide_z)
        
        if len(lstm_df) < 20:
            lstm_df.loc[len(lstm_df)] = np.append(np.array(temp["Open"].values[-1]), lstm_vector)
        else:
            lstm_df = lstm_df.drop(0).reset_index(drop=True)
            lstm_df.loc[len(lstm_df)] = np.append(np.array(temp["Open"].values[-1]), lstm_vector)
            lstm_df.to_csv("out_BTCUSDT.csv",index=False)
            
            !python replay.py
            
            with open(num_log, "rt") as f:
                a=f.readlines()
                
            pos=float(a[-1].split('\t')[0]) #가져야할 포지션
            pos_diff = pos - prev_pos
            
            cash = cash + lstm_df["Price"][19]*pos_diff
            asset = cash + lstm_df["Price"][19] * pos
            print("================================")
            print(lstm_df["Price"][19], asset, pos)
            print("================================")
            prev_pos = pos

            #reward 계산해야함
            

1/1 [==============================] - 0s 15ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.01 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.0 - Time 2022-07-26T15:21:32
Episode 5 - Step 18 - Epsilon 0.01 - Mean Reward -339.348 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.266 - Time 2022-07-26T15:21:33
Episode 10 - Step 18 - Epsilon 0.01 - Mean Reward 130.786 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.281 - Time 2022-07-26T15:21:33
34323.5 10000.0
1/1 [==============================] - 0s 15ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.01 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.015 - Time 2022-07-26T15:21:36
Episode 5 - Step 18 - Epsilon 0.01 - Mean Reward -270.294 - Mean Len

1/1 [==============================] - 0s 13ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.01 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.0 - Time 2022-07-26T15:22:07
Episode 5 - Step 18 - Epsilon 0.01 - Mean Reward -320.331 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.281 - Time 2022-07-26T15:22:07
Episode 10 - Step 18 - Epsilon 0.01 - Mean Reward -174.726 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.298 - Time 2022-07-26T15:22:07
34366.5 10000.0
1/1 [==============================] - 0s 15ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.01 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.016 - Time 2022-07-26T15:22:11
Episode 5 - Step 18 - Epsilon 0.01 - Mean Reward 255.996 - Mean Len

1/1 [==============================] - 0s 16ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.01 - Mean Reward -498.102 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.007 - Time 2022-07-26T15:22:49
Episode 5 - Step 18 - Epsilon 0.01 - Mean Reward 104.675 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.286 - Time 2022-07-26T15:22:50
Episode 10 - Step 18 - Epsilon 0.01 - Mean Reward 156.293 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.305 - Time 2022-07-26T15:22:50
34302.5 10000.0
1/1 [==============================] - 0s 16ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.01 - Mean Reward -966.219 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.0 - Time 2022-07-26T15:22:53
Episode 5 - Step 18 - Epsilon 0.01 - Mean Reward 994.906 - 

1/1 [==============================] - 0s 15ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.01 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.005 - Time 2022-07-26T15:23:20
Episode 5 - Step 18 - Epsilon 0.01 - Mean Reward 739.782 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.276 - Time 2022-07-26T15:23:20
Episode 10 - Step 18 - Epsilon 0.01 - Mean Reward 403.518 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.274 - Time 2022-07-26T15:23:21
Episode 15 - Step 18 - Epsilon 0.01 - Mean Reward 277.418 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.34 - Time 2022-07-26T15:23:21
Episode 20 - Step 18 - Epsilon 0.01 - Mean Reward 211.366 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.31 - Time 2022-07-26T15:23:21
Episode 25 - Step 18 - Epsilon 0.01 - Mean Reward 170.719 - Mean Lengt

1/1 [==============================] - 0s 14ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.01 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.016 - Time 2022-07-26T15:23:51
Episode 5 - Step 18 - Epsilon 0.01 - Mean Reward 1254.063 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.253 - Time 2022-07-26T15:23:51
Episode 10 - Step 18 - Epsilon 0.01 - Mean Reward 564.4 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.281 - Time 2022-07-26T15:23:51
Episode 15 - Step 18 - Epsilon 0.01 - Mean Reward 340.787 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.345 - Time 2022-07-26T15:23:52
Episode 20 - Step 18 - Epsilon 0.01 - Mean Reward 259.647 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.328 - Time 2022-07-26T15:23:52
Episode 25 - Step 18 - Epsilon 0.01 - Mean Reward 94.569 - Mean Lengt

...

1/1 [==============================] - 0s 16ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.001 - Mean Reward 1359.529 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.0 - Time 2022-07-26T17:31:21
Episode 5 - Step 18 - Epsilon 0.001 - Mean Reward 226.588 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.266 - Time 2022-07-26T17:31:22
Episode 10 - Step 18 - Epsilon 0.001 - Mean Reward 123.594 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.266 - Time 2022-07-26T17:31:22
33090.0 14909.355824500788
1/1 [==============================] - 0s 14ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.016 - Time 2022-07-26T17:31:25
Episode 5 - Step 18 - Epsilon 0.001 - Mean Reward

1/1 [==============================] - 0s 16ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.0 - Time 2022-07-26T17:32:03
Episode 5 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.266 - Time 2022-07-26T17:32:03
Episode 10 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.266 - Time 2022-07-26T17:32:03
33030.0 14909.355824500788
1/1 [==============================] - 0s 27ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.0 - Time 2022-07-26T17:32:06
Episode 5 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Len

1/1 [==============================] - 0s 15ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.0 - Time 2022-07-26T17:32:44
Episode 5 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.266 - Time 2022-07-26T17:32:45
Episode 10 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.344 - Time 2022-07-26T17:32:45
33090.5 14909.355824500788
1/1 [==============================] - 0s 10ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.0 - Time 2022-07-26T17:32:48
Episode 5 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Len

1/1 [==============================] - 0s 15ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.016 - Time 2022-07-26T17:33:26
Episode 5 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.25 - Time 2022-07-26T17:33:26
Episode 10 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.278 - Time 2022-07-26T17:33:27
33117.5 14909.355824500788
1/1 [==============================] - 0s 15ms/step
Loading model at checkpoints\2022-07-25T19-32-18\Trade_net_130.chkpt with exploration rate 0.01
Episode 0 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean Length 18.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 0.016 - Time 2022-07-26T17:33:30
Episode 5 - Step 18 - Epsilon 0.001 - Mean Reward 0.0 - Mean 

1/1 [==============================] - 0s 125ms/step


In [30]:
lstm_df["Price"]

0     34403.5
1     34407.5
2     34400.0
3     34350.5
4     34266.0
5     34316.0
6     34300.0
7     34350.5
8     34350.0
9     34349.0
10    34329.5
11    34300.5
12    34317.5
13    34325.5
14    34341.5
15    34305.0
16    34320.0
17    34365.0
18    34363.5
19    34323.5
Name: Price, dtype: float64

In [83]:
lstm_df

,Price,z0,z1,z2
0,34374.5,-1.985180,0.556164,2.426674
1,34379.5,-1.960876,0.568421,2.530124
2,34351.5,-1.930492,0.528077,2.216002


In [7]:
save_dir=Path("log")
num_log = save_dir / "num_log"
num_log

WindowsPath('log/num_log')

In [8]:
with open(num_log, "rt") as f:
    a=f.readlines()

0.0

In [84]:
float(a[3][:-1])

100.0